# Branching log analysis

In this Jupyter notebook we investigate what lead to the repeated instantiations terms.

In [ ]:
import axiom_profiler
parser = axiom_profiler.Parser('../z3-4.log')

## Compute the duplicated terms

In [ ]:
parser.initialise_repeated_instantiations()
duplicated_terms = parser.get_duplicated_terms()
duplicated_terms.sort()
print("Most common duplicated terms:")
for (count, term) in reversed(duplicated_terms[-100:]):
    print(count, term[:100])

## Investigate the most common term

In [ ]:
%pip install graphviz

In [ ]:
instantiations = parser.get_instantiations_for_term(duplicated_terms[-8][1])
print(len(instantiations))

In [ ]:
# Just pick the first one.
instantiation = instantiations[0]

In [ ]:
from graphviz import Digraph
from IPython.display import display

def render_dependencies(root_instantiation, max_depth=3, max_term_length=100):
    dot = Digraph()
    dot.node(str(root_instantiation))

    def walk(instantiation, current_depth):
        if current_depth >= max_depth:
            return
        offset = "  " * current_depth
        dot.node(str(instantiation))
        resulting_term = parser.render_instantiation_resulting_term(instantiation)
        print(offset + str(instantiation), resulting_term[:max_term_length])
        match = parser.get_instantiation_match(instantiation)
        blamed_report = parser.get_blamed_for_match(match)
        blamed_terms = blamed_report.get_blamed_terms()
        blamed_equalities = blamed_report.get_blamed_equalities()
        for blamed_term in blamed_terms + blamed_equalities:
            print(offset + "  " + parser.render_blamed_term(blamed_term))
            if blamed_term.has_created_by():
                print(offset + "    created by ", blamed_term.get_created_by())
                walk(blamed_term.get_created_by(), current_depth+1)
                dot.edge(str(instantiation), str(blamed_term.get_created_by()))
            else:
                print(offset + "    no created by ")

    walk(root_instantiation, 0)

    # Display the graph within the notebook
    display(dot)


In [ ]:
render_dependencies(instantiation, max_depth=5, max_term_length=10000)